In [1]:
# Step 0. Import libraries and custom modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Step 1. Load data, show info and sample
# 1.1 Read data
df_raw = pd.read_excel(
   'https://github.com/datagy/mediumdata/raw/master/sample_pivot.xlsx', 
   parse_dates=['Date']
)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1000 non-null   datetime64[ns]
 1   Region  1000 non-null   object        
 2   Type    1000 non-null   object        
 3   Units   911 non-null    float64       
 4   Sales   1000 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 39.2+ KB


In [5]:
# 1.2 Show sample
df_raw.sample(5, random_state=1234)

,Date,Region,Type,Units,Sales
681,2020-11-27,East,Children's Clothing,32.0,342
990,2020-12-17,North,Children's Clothing,7.0,217
155,2020-06-13,East,Children's Clothing,30.0,336
768,2020-06-25,North,Women's Clothing,5.0,644
438,2020-10-07,North,Women's Clothing,30.0,462


In [6]:
# Step 2. Transform to standard format
# 2.1 Create a pandas chain to convert to final form
df = (
    df_raw
    .copy()
    .set_axis(
        df_raw
        .columns.str.lower()
        .str.replace(' ','_')
        .str.replace(r'\W','',regex=True)
        .str.slice(0,40), axis=1
    )
    .astype({'region':'category', 
             'type':'category'})
    .assign(
        date = lambda x: pd.to_datetime(x['date'], format='%Y-%m-%d')
    )
    .set_index('date')
)

In [7]:
# Step 3. Perform busines analysis
# 3.1 Get the summary, for categorical and numerical columns

# Resumen de columnas numéricas
numerical_summary = df.describe()

# Resumen de columnas categóricas
categorical_summary = df.describe(include=['category'])

# Mostrar ambos resúmenes
print("Numerical Summary:")
print(numerical_summary)

print("\nCategorical Summary:")
print(categorical_summary)


Numerical Summary:
            units        sales
count  911.000000  1000.000000
mean    19.638858   427.254000
std      9.471309   253.441362
min      3.000000    33.000000
25%     12.000000   224.000000
50%     20.000000   380.000000
75%     28.000000   575.000000
max     35.000000  1155.000000

Categorical Summary:
       region              type
count    1000              1000
unique      4                 3
top      East  Women's Clothing
freq      411               424


In [13]:
print(df.columns)


Index(['region', 'type', 'units', 'sales'], dtype='object')


In [ ]:
# 3.2 Get the sum of total sales

total_sales_sum = df['sales'].sum()

print("Total Sales Sum:", total_sales_sum)




Total Sales Sum: 427254


In [ ]:
# 3.3 Get the average of sales

average_sales = df['sales'].mean() 

print("Average Sales:", average_sales)


Average Sales: 427.254


In [20]:
# 3.4 Get the sum of sales by region

sales_by_region = df.groupby('region', observed=True)['sales'].sum()

print("Sum of Sales by Region:")
print(sales_by_region)





Sum of Sales by Region:
region
East     167763
North    138700
South     59315
West      61476
Name: sales, dtype: int64


In [22]:
# 3.4 Get the average sales by region

average_sales_by_region = df.groupby('region', observed=True)['sales'].mean()
average_sales_by_region_df = average_sales_by_region.reset_index()
average_sales_by_region_df.columns = ['Region', 'Average Sales']

print("Average Sales by Region:")
print(average_sales_by_region_df)


Average Sales by Region:
  Region  Average Sales
0   East     408.182482
1  North     438.924051
2  South     432.956204
3   West     452.029412


In [ ]:
# 3.5 Get the sum, mean and count of sales by region


sales_stats_by_region = df.groupby('region', observed=True)['sales'].agg(['sum', 'mean', 'count'])

sales_stats_by_region.columns = ['Total Sales', 'Average Sales', 'Sales Count']

print("Sum, Mean, and Count of Sales by Region:")
print(sales_stats_by_region)


Sum, Mean, and Count of Sales by Region:
        Total Sales  Average Sales  Sales Count
region                                         
East         167763     408.182482          411
North        138700     438.924051          316
South         59315     432.956204          137
West          61476     452.029412          136


In [26]:
# 3.6 Create a pivot table that gets total units by type


pivot_units_by_type = df.pivot_table(values='units', index='type', aggfunc='sum', observed=True)

print("Total Units by Type:")
print(pivot_units_by_type)



Total Units by Type:
                      units
type                       
Children's Clothing  5887.0
Men's Clothing       3974.0
Women's Clothing     8030.0


In [27]:
# 3.7 Display the dataset showing only the East region

east_region_data = df[df['region'] == 'East']

print("Dataset for East Region:")
print(east_region_data)


Dataset for East Region:
           region                 type  units  sales
date                                                
2020-07-11   East  Children's Clothing   18.0    306
2020-02-28   East  Children's Clothing   26.0    832
2020-03-25   East     Women's Clothing   29.0    609
2020-11-03   East  Children's Clothing   34.0    374
2020-05-01   East       Men's Clothing   10.0    140
...           ...                  ...    ...    ...
2020-03-20   East       Men's Clothing   14.0    238
2020-06-05   East  Children's Clothing   11.0    220
2020-05-04   East       Men's Clothing    6.0    108
2020-10-18   East     Women's Clothing   19.0    399
2020-02-11   East  Children's Clothing   35.0    735

[411 rows x 4 columns]


In [28]:
# 3.8 Display the dataset showing only the East and West regions


east_west_region_data = df[df['region'].isin(['East', 'West'])]

print("Dataset for East and West Regions:")
print(east_west_region_data)


Dataset for East and West Regions:
           region                 type  units  sales
date                                                
2020-07-11   East  Children's Clothing   18.0    306
2020-02-28   East  Children's Clothing   26.0    832
2020-03-19   West     Women's Clothing    3.0     33
2020-03-25   East     Women's Clothing   29.0    609
2020-11-03   East  Children's Clothing   34.0    374
...           ...                  ...    ...    ...
2020-02-08   West       Men's Clothing   32.0    928
2020-05-04   East       Men's Clothing    6.0    108
2020-11-17   West       Men's Clothing   27.0    486
2020-10-18   East     Women's Clothing   19.0    399
2020-02-11   East  Children's Clothing   35.0    735

[547 rows x 4 columns]


In [29]:
# 3.9 Display the dataset showing East sales with units over 30 

east_sales_over_30_units = df[(df['region'] == 'East') & (df['units'] > 30)]

print("Dataset for East Region with Units Over 30:")
print(east_sales_over_30_units)


Dataset for East Region with Units Over 30:
           region                 type  units  sales
date                                                
2020-11-03   East  Children's Clothing   34.0    374
2020-06-12   East     Women's Clothing   35.0   1050
2020-05-11   East       Men's Clothing   35.0    700
2020-06-14   East       Men's Clothing   32.0    416
2020-10-28   East       Men's Clothing   34.0    986
...           ...                  ...    ...    ...
2020-02-24   East     Women's Clothing   32.0    693
2020-08-13   East       Men's Clothing   32.0    132
2020-10-04   East       Men's Clothing   35.0    350
2020-10-20   East       Men's Clothing   32.0    928
2020-02-11   East  Children's Clothing   35.0    735

[70 rows x 4 columns]


In [31]:
# 3.10 A data corruption has been detected. Add 1 to all units. 


df['units'] = df['units'] + 1

print("Updated Dataset with Units Corrected:")
print(df.sample(10))


Updated Dataset with Units Corrected:
           region                 type  units  sales
date                                                
2020-07-21   East     Women's Clothing    7.0    240
2020-08-01  North     Women's Clothing   20.0     57
2020-12-26  North       Men's Clothing    NaN    275
2020-05-16  South  Children's Clothing   30.0    529
2020-02-15  North     Women's Clothing    6.0    380
2020-08-06  South     Women's Clothing   28.0    546
2020-02-09  South     Women's Clothing   10.0    754
2020-12-17   East     Women's Clothing   28.0     60
2020-05-28   West     Women's Clothing   14.0    128
2020-03-03   East     Women's Clothing    8.0    432


In [32]:
# 3.11 Another issue was detected, in West region add 2 to all units

df.loc[df['region'] == 'West', 'units'] += 2


print("Updated Dataset with Units Corrected in West Region:")
print(df[df['region'] == 'West'].sample(5))


Updated Dataset with Units Corrected in West Region:
           region                 type  units  sales
date                                                
2020-02-14   West  Children's Clothing   34.0    910
2020-03-29   West  Children's Clothing   22.0    261
2020-12-19   West     Women's Clothing   13.0    800
2020-08-30   West  Children's Clothing   25.0     60
2020-11-17   West  Children's Clothing   27.0    990
